In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv("../dataset/regularite-mensuelle-tgv-aqst.csv" ,sep=",")
print(data.shape)
print(data.columns)

(8154, 26)
Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'commentaire_annulation',
       'nb_train_depart_retard', 'retard_moyen_depart',
       'retard_moyen_tous_trains_depart', 'commentaire_retards_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'commentaires_retard_arrivee',
       'nb_train_retard_sup_15', 'retard_moyen_trains_retard_sup15',
       'nb_train_retard_sup_30', 'nb_train_retard_sup_60',
       'prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic',
       'prct_cause_materiel_roulant', 'prct_cause_gestion_gare',
       'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')


In [3]:
data.head()

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
0,2018-01,National,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,110,6.511118,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,2018-01,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,26,7.548387,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000
3,2018-01,National,PARIS MONTPARNASSE,ST MALO,156,102,0,NaN,12,19.912500,...,8,6.724757,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000
4,2018-01,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,NaN,61,7.796995,...,17,3.346487,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000


# Estimation de retard

In [4]:
#on transforme en type date la colonne date pour appliquer des masks pandas plus facilement
data['date'] = pd.to_datetime(data['date']) 

#on transforme la donnée des gares en utilisant une méthode one-hot
liste_gare_depart = data['gare_depart'].unique()
liste_gare_arrivee = data['gare_arrivee'].unique()

data = pd.concat([data,pd.get_dummies(data['gare_depart'], prefix='gare_depart'),pd.get_dummies(data['gare_arrivee'], prefix='gare_arrivee')],axis = 1).drop(['gare_depart','gare_arrivee'],axis=1)

#on sépare en train set et test set en fonction de la date avant et après 2023
data_train = data[ data['date']<= '2022-12' ]
data_test = data[ data['date']>= '2023-01' ]

#on sépare les colonnes connues de la colonne a estimer
X_train = data_train.filter(items = ['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne','nb_train_prevu']+["gare_depart_"+i for i in liste_gare_depart]+["gare_arrivee_"+j for j in liste_gare_arrivee],axis = 1)
X_test = data_test.filter(items = ['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne','nb_train_prevu']+["gare_depart_"+i for i in liste_gare_depart]+["gare_arrivee_"+j for j in liste_gare_arrivee],axis = 1)
y_train = data_train['retard_moyen_arrivee']
y_test = data_test['retard_moyen_arrivee']

X_train.head()

,date,service,duree_moyenne,nb_train_prevu,gare_depart_BORDEAUX ST JEAN,gare_depart_LA ROCHELLE VILLE,gare_depart_PARIS MONTPARNASSE,gare_depart_QUIMPER,gare_depart_RENNES,gare_depart_ST PIERRE DES CORPS,...,gare_arrivee_DIJON VILLE,gare_arrivee_MADRID,gare_arrivee_ARRAS,gare_arrivee_AIX EN PROVENCE TGV,gare_arrivee_ANNECY,gare_arrivee_NICE VILLE,gare_arrivee_PERPIGNAN,gare_arrivee_SAINT ETIENNE CHATEAUCREUX,gare_arrivee_TOULON,gare_arrivee_GENEVE
0,2018-01-01,National,141,870,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,National,165,222,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,National,220,248,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,National,156,102,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,National,61,391,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
"""
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

cat_ordinal_features = ['date','service']
numerical_features = ['duree_moyenne','nb_train_prevu']

cat_ordinal_pipeline = make_pipeline(LabelEncoder(),MinMaxScaler())
numerical_pipeline = make_pipeline(MinMaxScaler())

preprocessor = make_column_transformer((cat_ordinal_pipeline, cat_ordinal_features),
                                       (numerical_pipeline, numerical_features))
"""

"\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.preprocessing import MinMaxScaler\n\nfrom sklearn.compose import make_column_transformer\nfrom sklearn.pipeline import make_pipeline\n\ncat_ordinal_features = ['date','service']\nnumerical_features = ['duree_moyenne','nb_train_prevu']\n\ncat_ordinal_pipeline = make_pipeline(LabelEncoder(),MinMaxScaler())\nnumerical_pipeline = make_pipeline(MinMaxScaler())\n\npreprocessor = make_column_transformer((cat_ordinal_pipeline, cat_ordinal_features),\n                                       (numerical_pipeline, numerical_features))\n"

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

scaler_date = MinMaxScaler()
scaler_date.fit(np.array(X_train['date'].dt.month).reshape(-1, 1))
X_train.loc[:,'date'] = scaler_date.transform(np.array(X_train['date'].dt.month).reshape(-1, 1))
X_test.loc[:,'date'] = scaler_date.transform(np.array(X_test['date'].dt.month).reshape(-1, 1))

encoder_service = LabelEncoder()
X_train.loc[:,'service'] = encoder_service.fit_transform(X_train['service'])
X_test.loc[:,'service'] = encoder_service.transform(X_test['service'])

scaler_duree_moyenne = MinMaxScaler()
X_train.loc[:,'duree_moyenne'] = scaler_duree_moyenne.fit_transform(np.array(X_train['duree_moyenne']).reshape(-1, 1))
X_test.loc[:,'duree_moyenne'] = scaler_duree_moyenne.transform(np.array(X_test['duree_moyenne']).reshape(-1, 1))

scaler_nb_train_prevu = MinMaxScaler()
X_train.loc[:,'nb_train_prevu'] = scaler_nb_train_prevu.fit_transform(np.array(X_train['nb_train_prevu']).reshape(-1, 1))
X_test.loc[:,'nb_train_prevu'] = scaler_nb_train_prevu.transform(np.array(X_test['nb_train_prevu']).reshape(-1, 1))

In [7]:
X_train.head()

,date,service,duree_moyenne,nb_train_prevu,gare_depart_BORDEAUX ST JEAN,gare_depart_LA ROCHELLE VILLE,gare_depart_PARIS MONTPARNASSE,gare_depart_QUIMPER,gare_depart_RENNES,gare_depart_ST PIERRE DES CORPS,...,gare_arrivee_DIJON VILLE,gare_arrivee_MADRID,gare_arrivee_ARRAS,gare_arrivee_AIX EN PROVENCE TGV,gare_arrivee_ANNECY,gare_arrivee_NICE VILLE,gare_arrivee_PERPIGNAN,gare_arrivee_SAINT ETIENNE CHATEAUCREUX,gare_arrivee_TOULON,gare_arrivee_GENEVE
0,0.0,1,0.179389,0.827783,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1,0.209924,0.211227,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1,0.279898,0.235966,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,1,0.198473,0.097050,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,1,0.077608,0.372027,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [9]:
model.fit(X_train,y_train)

LinearRegression()

In [10]:
y_predicted = model.predict(X_test)
print(y_predicted)

[33.70214844 40.39941406 37.35644531 34.68066406 31.28417969 23.54101562
 44.32324219 25.25390625 45.66699219 41.59667969 49.20117188 30.95800781
 26.93261719 20.52148438 35.85351562 40.32324219 19.13769531 28.41308594
 49.24316406 39.48339844 43.98339844 40.9140625  31.76855469 32.79003906
 37.40917969 18.54882812 37.77441406 22.36621094 34.08886719 37.0625
 43.57128906 47.13964844 33.61230469 38.57519531 37.60839844 42.52929688
 49.37890625 28.53808594 37.62695312 28.25       31.54003906 22.59277344
 31.86914062 37.61523438 38.15917969 38.81738281 42.52832031 46.90625
 21.88964844 46.86425781 28.35839844 32.40722656 24.32714844 32.37109375
 35.46191406 45.47363281 43.99902344 37.83691406 36.34472656 38.69140625
 36.31152344 33.89746094 45.37792969 35.72753906 20.78027344 26.68652344
 45.08984375 22.07324219 33.90136719 39.38867188 44.23730469 38.68652344
 50.00488281 35.39746094 31.41699219 39.09375    39.95410156 34.41503906
 46.20410156 37.24609375 29.61816406 36.80078125 43.762695

In [11]:
score = model.score(X_test,y_test)

print(score)

0.14414583611510423
